# Modelling v7

modelling use the latest exploration

# Goal

Get the model from the prepared data. With undersampling + sample weight. 

# Plan

Checklist what will be done on the notebook :

    [*] Get Data
    [*] Modelling



In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
import time

from scipy.stats import uniform,randint
import warnings
warnings.filterwarnings("ignore")

In [2]:
if 'iter' in os.getcwd().split('/')[-1]:
    ######## run manually #############
    with open("../../config.yaml", "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    if os.name == 'posix' :
        PATH_MODEL = config['path']['artifact']+os.getcwd().split('/')[-1]+'/'
    else:
        PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'

else:
    ######### run with makefile #############
    with open("config-make.yaml", "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    if os.name == 'posix' :
        PATH_MODEL = config['path']['artifact']+'iteration_2/'
    else:
        PATH_MODEL = config['path']['artifact']+'iteration_2/'

PATH_RAW = config['path']['raw']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']
PATH_UDF = config['path']['udf']

In [3]:
from sklearn.linear_model import LogisticRegression, ElasticNet, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessRegressor

## Get Data

In [4]:
raw_data = pd.read_csv(PATH_PROCESSED+'prep_iter2.csv')
col_cat_load = joblib.load(PATH_MODEL+'cat_names.pkl')
col_num_load = joblib.load(PATH_MODEL+'num_names.pkl')

In [5]:
all_tr = raw_data[raw_data.is_train == 1]
down_size = int(len(all_tr[all_tr.target == 1]) * 3.5)
all_tr0 = all_tr[all_tr.target == 0].sample(down_size,random_state = 125)

all_tr0['sweight'] = 2
all_tr1 = all_tr[all_tr.target == 1]
all_tr1['sweight'] = 1

all_tr_down = pd.concat([all_tr0,all_tr1])
all_tr_down.target.value_counts(normalize = 1)

samp_w = all_tr_down['sweight']

In [6]:
X_tr = all_tr_down[col_num_load+col_cat_load]
Y_tr = all_tr_down['target'].values

X_val = raw_data[raw_data.is_train == 0][col_num_load+col_cat_load]
Y_val = raw_data[raw_data.is_train == 0]['target'].values

In [7]:
X_tr.shape

(16974, 26)

In [8]:
X_val.shape

(5765, 26)

In [9]:
Y_val.shape

(5765,)

In [10]:
Y_tr.mean()

0.2222222222222222

In [11]:
samp_w

5079     2
17291    2
9206     2
4581     2
19666    2
        ..
38422    1
38424    1
38425    1
38426    1
38427    1
Name: sweight, Length: 16974, dtype: int64

## Modelling

In [12]:
proportion_cls = pd.Series(Y_tr).value_counts(normalize = True)
scale_pos_w = np.ceil(proportion_cls[0]/proportion_cls[1])
# scale_pos_w = 
cw = {0:1, 1:scale_pos_w}
cw
# scale_pos_weight for xgboost

{0: 1, 1: 4.0}

In [13]:
# proportion_cls

In [14]:
base_logreg = LogisticRegression(class_weight=cw)
all_model = {'sgd7' : SGDClassifier(class_weight=cw,max_iter = 5500,early_stopping=True,eta0 = 0.15)
            ,'logreg7' : LogisticRegression(class_weight=cw)
            ,'nb7': GaussianNB(priors=[0.78,0.22])
            ,'bagtree7': BaggingClassifier(random_state=234)
            ,'baglog7': BaggingClassifier(base_estimator=base_logreg, random_state=234)
            ,'adab_tree7':AdaBoostClassifier(random_state=234)
            ,'adab_log7':AdaBoostClassifier(base_estimator=base_logreg,random_state=234)
            }

In [15]:
param_options = {'logreg7' : dict(C=uniform(loc=0, scale=2)
                                 ,penalty=['l2', 'l1']
                                 ,solver=['liblinear', 'sag'])
                 ,'nb7' : dict(var_smoothing = uniform(loc=1e-8, scale=1e-2))
                 ,'sgd7' : dict(loss=['hinge', 'perceptron']
                                ,penalty=['l2', 'l1']
                                ,alpha=uniform(loc=0.01, scale=0.9)
                                ,l1_ratio=uniform(loc=0.01, scale=0.8)
                                ,learning_rate=['constant','optimal','invscaling','adaptive'])
                 ,'bagtree7' : dict(n_estimators=randint(4, 35)
                                   ,max_samples=uniform(loc=0.2,scale=0.65)
                                   ,max_features=uniform(loc=0.2,scale=0.65)
                                   ,warm_start=[False,True])
                 ,'baglog7' : dict(n_estimators=randint(4, 35)
                                   ,max_samples=uniform(loc=0.2,scale=0.65)
                                   ,max_features=uniform(loc=0.2,scale=0.65)
                                   ,warm_start=[False,True])
                 ,'adab_tree7' : dict(n_estimators=randint(50, 120)
                                   ,learning_rate=uniform(loc=0.05,scale=0.8)
                                  )
                 ,'adab_log7' : dict(n_estimators=randint(5, 50)
                                   ,learning_rate=uniform(loc=0.05,scale=0.65)
                                  )
                }

 # min_weight_fraction_leaf must in [0, 0.5]

In [1]:
## scoring
## https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
best_clf = {}
best_scr = {}
sla_model = {}
for mdl in all_model.keys():
    L1 = time.time()
    clf = all_model[mdl]
    parm = param_options[mdl]
    RS = RandomizedSearchCV(clf, parm, n_iter = 200, cv = 5, scoring = 'roc_auc',n_jobs = 10)
        
    RS.fit(X_tr,Y_tr, sample_weight=samp_w)
    
    best_clf[mdl] = RS.best_estimator_
    best_scr[mdl] = RS.best_score_
    diff_time = round((time.time()-L1)/60, 2)
    
    joblib.dump(best_clf[mdl],PATH_MODEL+mdl+'.pkl')
    
    print(mdl, 'is done in ',diff_time, ' minutes')
    sla_model[mdl] = diff_time

In [17]:
sla_model

{'sgd7': 0.38,
 'logreg7': 2.57,
 'nb7': 0.13,
 'bagtree7': 0.85,
 'baglog7': 4.87,
 'adab_tree7': 2.8,
 'adab_log7': 3.06}

In [18]:
best_scr

{'sgd7': 0.83703990586309,
 'logreg7': 0.8354104484473641,
 'nb7': 0.7610665094603093,
 'bagtree7': 0.8284644856795029,
 'baglog7': 0.8359857938461215,
 'adab_tree7': 0.8468077080006211,
 'adab_log7': 0.8017777274862464}

In [19]:
best_clf

{'sgd7': SGDClassifier(alpha=0.010257240430073535, class_weight={0: 1, 1: 4.0},
               early_stopping=True, eta0=0.15, l1_ratio=0.41527835874567165,
               max_iter=5500),
 'logreg7': LogisticRegression(C=0.022301288998292312, class_weight={0: 1, 1: 4.0},
                    penalty='l1', solver='liblinear'),
 'nb7': GaussianNB(priors=[0.78, 0.22], var_smoothing=9.23202551413979e-05),
 'bagtree7': BaggingClassifier(max_features=0.591459873595623,
                   max_samples=0.4353495541259752, n_estimators=34,
                   random_state=234, warm_start=True),
 'baglog7': BaggingClassifier(base_estimator=LogisticRegression(class_weight={0: 1,
                                                                   1: 4.0}),
                   max_features=0.3085758517738, max_samples=0.6362809678412202,
                   n_estimators=17, random_state=234, warm_start=True),
 'adab_tree7': AdaBoostClassifier(learning_rate=0.685075433575406, n_estimators=119,
           

## Evaluation

The SVM is best so far, but there is no predict proba.

In [20]:
for ml in best_clf.keys():
    Y_prob_pred = best_clf[ml].predict(X_val)
    print(ml)
    print(classification_report(Y_val,Y_prob_pred))

sgd7
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      5079
           1       0.41      0.68      0.51       686

    accuracy                           0.84      5765
   macro avg       0.68      0.77      0.71      5765
weighted avg       0.89      0.84      0.86      5765

logreg7
              precision    recall  f1-score   support

           0       0.95      0.88      0.91      5079
           1       0.42      0.65      0.51       686

    accuracy                           0.85      5765
   macro avg       0.69      0.76      0.71      5765
weighted avg       0.89      0.85      0.87      5765

nb7
              precision    recall  f1-score   support

           0       0.94      0.77      0.85      5079
           1       0.28      0.65      0.39       686

    accuracy                           0.76      5765
   macro avg       0.61      0.71      0.62      5765
weighted avg       0.86      0.76      0.80      5765

ba

In [21]:
## Save train data for explainability
joblib.dump(X_tr,PATH_MODEL+'train_data7.pkl')
joblib.dump(Y_tr,PATH_MODEL+'target_data7.pkl')

['artifacts/iteration_2/target_data7.pkl']

In [22]:
1-Y_val.mean()

0.8810060711188205